In [2]:
#load the saved model and check predictions
from keras.models import Model, load_model
from keras.layers import Input
import numpy as np
import csv

In [3]:
#form input_texts
input_texts = []
with open('test_data(copy).csv') as csvfile:
    next(csvfile)
    readCSV = csv.reader(csvfile,delimiter=',')
    for row in readCSV:
        text = " ".join(row)
        input_texts.append(text)
#input_texts

In [4]:
#form output_texts
with open('description.txt','r',encoding='utf-8') as f:
    lines =   f.read().split('\n') 
target_texts = []
for line in lines:
    line = 'START_ ' + line + ' _END'
    target_texts.append(line)
#target_texts

In [5]:
input_words = set()
target_words = set()
for input_text,target_text in zip(input_texts,target_texts):
    words = input_text.split(' ') 
    for word in words:
        if word not in input_words:
            input_words.add(word)
    words = target_text.split(' ') 
    for word in words:
        if word not in target_words:
            target_words.add(word)
input_words = sorted(list(input_words))
target_words = sorted(list(target_words))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
input_text_sizes = []
for text in input_texts:
    input_text_sizes.append(len(text.split(' ')))
target_text_sizes = []
for text in target_texts:
    target_text_sizes.append(len(text.split(' ')))
max_encoder_seq_length = max(input_text_sizes)
max_decoder_seq_length = max(target_text_sizes)  

In [6]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

In [7]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length),
    dtype='float32')
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t, word in enumerate(input_text.split(' ')):
        encoder_input_data[i,t]=input_token_index[word]

In [8]:
model = load_model('model.h5')
model.summary()







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     505600      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 50)     506450      input_2[0][0]                    
_________

In [9]:
encoder_inputs = model.input[0]
encoder_outputs,state_h_enc,state_c_enc = model.layers[4].output
encoder_states = [state_h_enc,state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

In [11]:
decoder_state_input_h = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_inputs = model.input[1]

dex=  model.layers[3]

final_dex2= dex(decoder_inputs)

decoder_lstm = model.layers[5]
decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_dense = model.layers[6]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [12]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 200):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [14]:
for seq_index in range(1000,1010):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: ['1001 Credit 811 India US USD 0 0.12']
Decoded sentence:  Transaction no 1001 is a credit transaction, from India to US worth 811 dollars, which is suspicious with 0.12 probability. _END
-
Input sentence: ['1002 Credit 6564 Pakistan Germany Euro 1 0.7']
Decoded sentence:  Transaction no 1002 is a credit transaction, from Pakistan to Germany worth 6564 euros, which is not suspicious with 0.7 probability. _END
-
Input sentence: ['1003 Credit 606 India India INR 0 0.95']
Decoded sentence:  Transaction no 1003 is a credit transaction, within India worth 606 ruppees, which is suspicious with 0.95 probability. _END
-
Input sentence: ['1004 Debit 671 Pakistan Japan Euro 0 0.38']
Decoded sentence:  Transaction no 1004 is a debit transaction, from Pakistan to Japan worth 671 euros, which is suspicious with 0.38 probability. _END
-
Input sentence: ['1005 Debit 7750 US India INR 0 0.28']
Decoded sentence:  Transaction no 1005 is a debit transaction, from US to India worth 7750 